# A notebook to illustrate how to compute and correct Impulse Response in IW SLC product

## compute IW1 impulse response

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from xsarslc.processing.impulseResponse import generate_IWS_AUX_file_ImpulseReponse
# SENTINEL1_DS:/home/datawork-cersat-public/cache/project/mpc-sentinel1/data/esa/sentinel-1a/L1/WV/S1A\_WV\_SLC\_\_1S/2018/050/S1A\_WV\_SLC\_\_1SSV\_20180219T221522\_20180219T222851\_020681\_0236CE\_8F55.SAFE:WV_051
subswathes = {'/home/datawork-cersat-public/cache/project/mpc-sentinel1/data/esa/sentinel-1a/L1/IW/S1A_IW_SLC__1S/2022/271/S1A_IW_SLC__1SDV_20220928T095555_20220928T095622_045203_056722_FB4F.SAFE/':[1],
             }
subswath_number = 1

IRs_IW1_VV = generate_IWS_AUX_file_ImpulseReponse(subswathes, subswath_number , polarization='VV')
IRs_IW1_VV
IRs_IW1_VH = generate_IWS_AUX_file_ImpulseReponse(subswathes, subswath_number , polarization='VH')
IRs_IW1_VH

## save IR file to netCDF

In [ ]:
out_file = "/tmp/S1_IR_IW1_VV_AUX_FILE.nc"
if 'footprint' in IRs_IW1_VV.attrs:
    IRs_IW1_VV.attrs.update({'footprint': str(IRs_IW1_VV.footprint)})
if 'multidataset' in IRs_IW1_VV.attrs:
    IRs_IW1_VV.attrs.update({'multidataset': str(IRs_IW1_VV.multidataset)})
IRs_IW1_VV.to_netcdf(out_file)
print(out_file)

## display IW1 range impulse response for VV and VH

In [ ]:
from matplotlib import pyplot as plt
IRs_IW1_VV['range_IR'].plot(label='VV')
IRs_IW1_VH['range_IR'].plot(label='VH',alpha=0.7)
plt.legend()
plt.grid(True)
plt.title('burst = %s subswath = %s'%(IRs_IW1_VH.burst.values,IRs_IW1_VH.attrs['subswath']))

## display IW1 azimuth impulse response for VV and VH

In [ ]:
from matplotlib import pyplot as plt
IRs_IW1_VV['azimuth_IR'].plot(label='VV')
IRs_IW1_VH['azimuth_IR'].plot(label='VH',alpha=0.7)
plt.legend()
plt.grid(True)
plt.title('burst = %s subswath = %s'%(IRs_IW1_VH.burst.values,IRs_IW1_VH.attrs['subswath']))

## apply IR correction when computing a cross spectra

In [ ]:
import xsarslc
import xsar
import os
one_tiff = '/home/datawork-cersat-public/cache/project/mpc-sentinel1/data/esa/sentinel-1a/L1/IW/S1A_IW_SLC__1S/2022/127/S1A_IW_SLC__1SDV_20220507T162411_20220507T162439_043107_0525DE_734D.SAFE/measurement/s1a-iw1-slc-vv-20220507t162411-20220507t162439-043107-0525de-004.tiff'
fullpathsafeSLC = os.path.dirname(os.path.dirname(one_tiff))
imagette_number = os.path.basename(one_tiff).split('-')[1].replace('iw','')
str_gdal = 'SENTINEL1_DS:%s:IW%s'%(fullpathsafeSLC,imagette_number)
print('str_gdal',str_gdal)
xsarobj = xsar.Sentinel1Dataset(str_gdal) #,resolution='30m'
xsarobj

In [ ]:
dt = xsarobj.datatree
dt.load()
dt

In [ ]:
import xsarslc.processing.xspectra
intra_xs_without_IR = xsarslc.processing.xspectra.compute_IW_subswath_intraburst_xspectra(dt, 
                                         tile_width={'sample': 20.e3, 'line': 20.e3},
                                         tile_overlap={'sample': 10.e3, 'line': 10.e3},
                                         periodo_width = {'sample': 4000, 'line': 4000},
                                        periodo_overlap = {'sample': 2000, 'line': 2000},
                                        polarization='VV',dev=True)
intra_xs_without_IR

In [ ]:
IR_path = out_file
intra_xs_with_IR = xsarslc.processing.xspectra.compute_IW_subswath_intraburst_xspectra(dt, 
                                         tile_width={'sample': 20.e3, 'line': 20.e3},
                                         tile_overlap={'sample': 10.e3, 'line': 10.e3},
                                         periodo_width = {'sample': 4000, 'line': 4000},
                                        periodo_overlap = {'sample': 2000, 'line': 2000},
                                        polarization='VV',dev=True,IR_path=IR_path)
intra_xs_with_IR

In [ ]:
bursti = 0
tile_line_i = 0
tile_sample_i = 0
onespec_without_IR = intra_xs_without_IR['xspectra_2tau'].isel({'burst':bursti,'tile_sample':tile_sample_i,'tile_line':tile_line_i}).squeeze()
onespec_without_IR = onespec_without_IR.swap_dims({'freq_line': 'k_az', 'freq_sample': 'k_rg'})
abs(onespec_without_IR.real.mean(dim='k_rg')).plot(label='real without IR')

onespec_with_IR = intra_xs_with_IR['xspectra_2tau'].isel({'burst':bursti,'tile_sample':tile_sample_i,'tile_line':tile_line_i}).squeeze()
onespec_with_IR = onespec_with_IR.swap_dims({'freq_line': 'k_az', 'freq_sample': 'k_rg'})
abs(onespec_with_IR.real.mean(dim='k_rg')).plot(label='real with IR')
plt.legend()
plt.grid(True)

In [ ]:
onespec_without_IR.imag.mean(dim='k_rg').plot(label='imag. without IR')
onespec_with_IR.imag.mean(dim='k_rg').plot(label='imag. with IR')
plt.legend()
plt.grid(True)